# Steps:
* 1) Collect all missing comment IDs in files for 2015 compiled by @dgaff
* 2) Collect all parent IDs that were referenced by comments in 2015, from pushshift.io data dumps on BigQuery
* 3) Intersecting missing comment IDs with referenced BQ parent IDs - gives us list of missing parents ("dangling references")
* 4) Patch step: Using the Reddit API, query for missing parents and measure the amount we can retrieve.

Note: This analysis works ONLY WITH COMMENTS from 2015. Numbers for missing 'reference' submissions would change. But for the current analysis, only missing 'referenced' comments are considered.

In [115]:
'''
author: eshwar_chan
date: 15-03-2018
'''

import pandas as pd

filepath = "monthly_missing_data_start_to_2017_06/2015/"

monthly_dangling_references = []

overall_parent_ids_2015 = 0
overall_missing_parents_2015 = 0

for month in range(1,13):

    import time
    start_time = time.time()

    if month < 10:
        month_str = "0" + str(month)
    else:
        month_str = str(month)
    
    print("Working on ", month_str)
    
    df = pd.read_csv(filepath + 'RC_2015-' + month_str + '.csv', names = ['start_missing_ID', 'next_known_ID', 'gap_size'])
    
    list_missing_IDs = []

    for index, row in df.iterrows():
        start = row['start_missing_ID'].astype(int)
        offset = row['gap_size']

        for i in range(start, start + offset):
            list_missing_IDs.append(i)

    temp = pd.DataFrame()
    temp['name'] = list_missing_IDs

    from numpy import base_repr

    def base_10_to_36(num):
        return base_repr(num, 36).lower()

    temp['name'] = temp['name'].apply(base_10_to_36)
    
    ###get IDs in reddit parent_ID format; example: <t1_XXXX>
    comment_ids = []
    for _id in temp['name']:
        comment_ids.append("t1_" + str(_id))
    temp['comment_id'] = comment_ids

    ###load all parent_IDs from BQ
    bigquery_parent_IDs = pd.read_csv('pid-2015-' + month_str + '.dms')
    
    ###intersection between missing IDs and parent IDs (comments only)
    hit_list = bigquery_parent_IDs[bigquery_parent_IDs.parent_id.isin(temp['comment_id'])]
    monthly_dangling_references.append(hit_list)
    print("No. of missing parent IDs for month 2015-", month_str ," = ", len(hit_list))
    
    ###print them out
    import sys
    for _id in hit_list['parent_id']:
        sys.stdout.write("\"" + str(_id) + "\",")
    
    overall_missing_parents_2015 += len(hit_list)
    overall_parent_ids_2015 += len(bigquery_parent_IDs)
    
    print("\n--- %s seconds since start of this month's analysis---" % (time.time() - start_time))

print("Total parents in 2015: ", overall_parent_ids_2015)
print("Total missing parents in 2015: ", overall_missing_parents_2015)

Working on  01
No. of missing parent IDs for month 2015- 01  =  21
"t1_co3ontf","t1_cnyey08","t1_cnd04s1","t1_co3z58a","t1_cny5yke","t1_cnpb6kp","t1_cnc3slo","t1_cnyfhnc","t1_cnd2ioz","t1_cniq8lu","t1_cnbv877","t1_cnz5i1m","t1_cnm8sgx","t1_co3etpr","t1_cnmayc0","t1_cnxmsit","t1_cnrm1z9","t1_cnyf57p","t1_cnmzoli","t1_cnbvndi","t1_co2bfuj",
--- 58.25500273704529 seconds since start of this month's analysis---
Working on  02
No. of missing parent IDs for month 2015- 02  =  132
"t1_copf7ae","t1_coprx8k","t1_cop8gbt","t1_coh2l74","t1_copmgsh","t1_copphsh","t1_copcdm8","t1_cov41g1","t1_copp7ph","t1_cohftft","t1_cobxhvl","t1_copmsou","t1_coplxl3","t1_cools54","t1_coj3o3v","t1_coh46si","t1_cognlh1","t1_copioz2","t1_coh1k8a","t1_copgrt4","t1_cod27qe","t1_copr1t1","t1_cov70r8","t1_copn2vd","t1_copmjie","t1_copmpj1","t1_coprlb1","t1_coph2t0","t1_cov1apg","t1_cohao66","t1_coppmia","t1_coj7ra2","t1_coplugs","t1_copesmv","t1_coo4htt","t1_copi1v8","t1_cophqe9","t1_copfq3d","t1_cov52gv","t1_coh0nry","

No. of missing parent IDs for month 2015- 06  =  898
"t1_crzwzx3","t1_cs7n288","t1_csld41w","t1_cs76yxj","t1_crwgtlp","t1_crzvwly","t1_crzvqze","t1_cs6wm1j","t1_cs6gz8v","t1_cs4hfzd","t1_crudsa0","t1_csegetj","t1_crzxcbb","t1_crxjpvv","t1_cs3835t","t1_cs3irpl","t1_cs9g4hx","t1_cslm7ry","t1_crzvgfe","t1_cs2zbdr","t1_cs4p7p2","t1_crxnbiz","t1_crzx6hx","t1_crzvp7v","t1_crzv56g","t1_crus103","t1_csevixp","t1_cs1jc0a","t1_crxegw6","t1_crytj1d","t1_cru3hhb","t1_csjq0vl","t1_cryvyhp","t1_crzw9h3","t1_cs5m4ec","t1_crzw603","t1_crzw6kg","t1_crse6aa","t1_crzvyaf","t1_cs2ipx9","t1_cryn2vz","t1_cs2ovxb","t1_cs9j14p","t1_cs4pgij","t1_crskr8y","t1_cs2fcvb","t1_cs7mk5h","t1_cs2zaxt","t1_crsx6u0","t1_cs5kc32","t1_crzvysm","t1_crrztkl","t1_cs3fw79","t1_cs38ph4","t1_cs3j9p0","t1_cry0ll5","t1_csk33ya","t1_csh7tm2","t1_cs6wkjl","t1_cs2hbp7","t1_crzwlvs","t1_cs3m837","t1_cs2q3k1","t1_crywepl","t1_csfvjjw","t1_cryzy55","t1_cs7o5l1","t1_crzwgh7","t1_cs4f0li","t1_crwa37o","t1_crzwyqd","t1_cs5ektq","t1_crzx977

No. of missing parent IDs for month 2015- 07  =  4504


"t1_csvu2id","t1_ctc98c2","t1_csu26xb","t1_ct1kue6","t1_ct40xdw","t1_csuz4jw","t1_ct71su5","t1_ctbnw6e","t1_ctcphba","t1_ctc4234","t1_csogncz","t1_ctcqi3j","t1_ct13xak","t1_csz7cyy","t1_ct2ygmr","t1_csrlbgz","t1_csqfevg","t1_ct2hbvg","t1_ct1o2qz","t1_ct7r8zz","t1_cswhub9","t1_ct97xgv","t1_ctbbt2s","t1_ct48i1k","t1_ctf7p8q","t1_ctd0xz4","t1_ctbq9dj","t1_ct93yun","t1_ctajbz6","t1_ct2yilq","t1_ctcdds0","t1_ct7e7v2","t1_csuyp99","t1_ct7it85","t1_ctbmrsx","t1_ct4xxro","t1_csvubxp","t1_ct9babl","t1_ct2bn72","t1_cst2kxd","t1_csy3zs9","t1_cstivrn","t1_ct1m835","t1_ct3f4hr","t1_cswbcaq","t1_csppqy0","t1_ct1d8f2","t1_ctdu0il","t1_ct5vkpt","t1_cspbp10","t1_ctbfwra","t1_ct92qvl","t1_ct9icg0","t1_ct4ji79","t1_ct7y7qd","t1_ct4abym","t1_cszidb6","t1_ct4cglg","t1_ct5wkvi","t1_csun70e","t1_ctcj4c0","t1_ct9nu54","t1_ct337pn","t1_cszrmlq","t1_ctcsi36","t1_ct3lp4l","t1_csqsc2l","t1_ct0cwri","t1_ct0lzyl","t1_csqfwpo","t1_ct90h2p","t1_ct8d7gq","t1_cstf7wz","t1_csynudu","t1_ct4qtp9","t1_cst9hmt","t1_csv99h1"

No. of missing parent IDs for month 2015- 08  =  81
"t1_ctp9ml1","t1_cu1e8w6","t1_cu20vdx","t1_ctous48","t1_cuhng6n","t1_cu1t5ro","t1_cuhf2sy","t1_cuhbr78","t1_cu6qusa","t1_ctr1ob4","t1_cu3h2q6","t1_cu42zvb","t1_cu0p9kl","t1_cu50awv","t1_ctwmwtw","t1_cua3ejs","t1_ctxgi03","t1_cuhnwnk","t1_ctwmwwn","t1_cu010z3","t1_ctwtc8j","t1_ctzj4og","t1_cuk5bq4","t1_ctwj83n","t1_ctzejb1","t1_ctwn3jx","t1_ctxgge1","t1_ctzceuo","t1_ctwn3h7","t1_cu9tjc3","t1_cu4669q","t1_cuhcgm8","t1_cuhpq5b","t1_cukales","t1_ctzu2q9","t1_ctzyups","t1_ctpon6k","t1_ctsfy77","t1_cu01eio","t1_cto6w8w","t1_cuium59","t1_cu8wjoq","t1_ctwh3zz","t1_cu607px","t1_cu00zmu","t1_cu80whb","t1_cua3fa2","t1_cu1vhk9","t1_cu1ut7s","t1_cuhlgpr","t1_cu3h381","t1_ctzlio6","t1_cu01j44","t1_cu19s0y","t1_cu0qg16","t1_ctr4mh3","t1_cu9khiv","t1_ctnybkf","t1_cu3erwg","t1_cu1e9ct","t1_cuhnju2","t1_cugpxgu","t1_cuju9ku","t1_cu1c9im","t1_ctzg8vn","t1_cu11jxk","t1_ctu7169","t1_cu8c4dv","t1_cu3h2q7","t1_cugz7ez","t1_ctzivq8","t1_cuj26qm","t1_ctzoyjm"

No. of missing parent IDs for month 2015- 12  =  335
"t1_cy6v94z","t1_cy1h0op","t1_cxs5vu5","t1_cxzdgkm","t1_cy79jl0","t1_cy1tpss","t1_cy6z6dr","t1_cy77h67","t1_cxlyvlz","t1_cy7d5vu","t1_cy79sk8","t1_cy7cnlq","t1_cy75fpy","t1_cy7cpuo","t1_cy1mii7","t1_cy77i3i","t1_cxwbz6y","t1_cy74xii","t1_cy4hm22","t1_cy76xek","t1_cy5zriv","t1_cxze5hp","t1_cy6zu7i","t1_cxt2g0y","t1_cxxf64z","t1_cy1s1gb","t1_cxynrrv","t1_cy0014b","t1_cy76u9v","t1_cy6l10u","t1_cy797dj","t1_cy73z1n","t1_cy74ag0","t1_cy3d631","t1_cy6tswx","t1_cy7bxyn","t1_cy38yla","t1_cy7crs0","t1_cxylijt","t1_cy7d42d","t1_cy772hl","t1_cy7dekm","t1_cy6sgkw","t1_cy7dbyx","t1_cy6li8o","t1_cxyywqo","t1_cy6zpx9","t1_cxzbpfi","t1_cy6jhez","t1_cy7919b","t1_cy1n516","t1_cy1wdcm","t1_cxxgbba","t1_cy5yeei","t1_cy733wm","t1_cy7cubk","t1_cy76j7y","t1_cxrx14e","t1_cxjes1u","t1_cy76tqt","t1_cxyhkdl","t1_cy5q0tx","t1_cy6gk9u","t1_cxo4wab","t1_cy6d4iw","t1_cy6l6pt","t1_cy7dbgk","t1_cy7an0b","t1_cy7cndo","t1_cy7aeea","t1_cy6q2rg","t1_cy6uuxv","t1_cy6k3so

In [117]:
print("Total parents in 2015: ", overall_parent_ids_2015)
print("Total missing parents in 2015: ", overall_missing_parents_2015)

Total parents in 2015:  331831351
Total missing parents in 2015:  7093


In [142]:
print("Percentage of references to missing comment IDs = ", 100*(overall_missing_parents_2015/overall_parent_ids_2015), " %")

Percentage of references to missing comment IDs =  0.002137531604118985  %


In [123]:
month = 1
for hit_list in monthly_dangling_references:
    print("Month = ", month, "/2015; #missing parents = ", len(hit_list))
    month +=1

Month =  1 /2015; #missing parents =  21
Month =  2 /2015; #missing parents =  132
Month =  3 /2015; #missing parents =  208
Month =  4 /2015; #missing parents =  138
Month =  5 /2015; #missing parents =  150
Month =  6 /2015; #missing parents =  898
Month =  7 /2015; #missing parents =  4504
Month =  8 /2015; #missing parents =  81
Month =  9 /2015; #missing parents =  24
Month =  10 /2015; #missing parents =  378
Month =  11 /2015; #missing parents =  224
Month =  12 /2015; #missing parents =  335


# Of 331M referenced parent comment IDs in 2015, 7093 are missing parents! 
* This is 0.0021 % out of all parent comments in 2015. This percentage would be smaller if we compare it to the actual number of comments in 2015, since this would include parents and non-parents (i.e., children or leaf nodes)

In [133]:
import praw
from itertools import islice
                
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     password=password,
                     user_agent=user_agent,
                     username = username)

# Query the Reddit API for these missing parent IDs and calculate how many comments can be retrieved (aka, "the patch step")

In [166]:
month = 1

missing_parents = 0
parents_retrieved = 0

for hit_list in monthly_dangling_references:
    print("Month = ", month, "/2015; #missing parents = ", len(hit_list))
    month +=1
    missing_ids = hit_list['parent_id']
    # fetch missing things by id
    missing_things = list( reddit.info(list(missing_ids)))
    
    missing_parents += len(missing_ids)
    parents_retrieved += len(missing_things)

    print('retrieved', len(missing_things), '/', len(missing_ids), 'things')
    
    if month < 10:
        month_str = "0" + str(month)
    else:
        month_str = str(month)

    ###write patches to file
    import json
    for thing in missing_things:
        data = json.dumps({i:str(j) for i, j in thing.__dict__.items() if not i.startswith('_')})    
        with open("patched-parentIDs-2015-" + month_str + ".txt", 'a+') as outfile:
            json.dump(data, outfile)

Month =  1 /2015; #missing parents =  21
retrieved 13 / 21 things
Month =  2 /2015; #missing parents =  132
retrieved 117 / 132 things
Month =  3 /2015; #missing parents =  208
retrieved 93 / 208 things
Month =  4 /2015; #missing parents =  138
retrieved 88 / 138 things
Month =  5 /2015; #missing parents =  150
retrieved 85 / 150 things
Month =  6 /2015; #missing parents =  898
retrieved 653 / 898 things
Month =  7 /2015; #missing parents =  4504
retrieved 653 / 4504 things
Month =  8 /2015; #missing parents =  81
retrieved 55 / 81 things
Month =  9 /2015; #missing parents =  24
retrieved 20 / 24 things
Month =  10 /2015; #missing parents =  378
retrieved 359 / 378 things
Month =  11 /2015; #missing parents =  224
retrieved 213 / 224 things
Month =  12 /2015; #missing parents =  335
retrieved 334 / 335 things


In [147]:
print("Overall missing parents in 2015: ", missing_parents)
print("Number of parents retrieved by patch step: ", parents_retrieved)
print("Percentage of missing parents that can be retrieved by the patch step = ", 100*(parents_retrieved/missing_parents) , " %")

Overall missing parents in 2015:  7093
Number of parents retrieved by patch step:  2683
Percentage of missing parents that can be retrieved by the patch step =  37.82602565910052  %


# Summary: 
* Out of 331M referenced parent comment IDs in 2015, 7093 are missing parents. 
* This is 0.0021 % out of all parent comments in 2015 (would be smaller when compared to total # of all comments in 2015).
* Out of these missing (parent) comments, 2683 could be retrieved by querying the Reddit API. 
* Inclusion of a patch step would help retrieve 37.83% of the above mentioned missing parent comments. 